<a href="https://colab.research.google.com/github/AhmedZ04/MachineLearning-Demos/blob/main/Copy_of_ass7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Enable widgets
from google.colab import output
output.enable_custom_widget_manager()
import ipywidgets as widgets

# Install otter-grader
%pip install -q otter-grader

# Download the tests directory from the course website (this will be used by otter-grader)
!wget -q https://vladtkachuk4.github.io/machinelearning1/assignments/Fall2024/ass7/tests.zip -O tests.zip

# Unzip the tests directory, forcing overwriting of existing files
!unzip -qo tests.zip -d .

In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook()

# CMPUT 267 - Machine Learning I
# Assignment 7 - Classification

## Assignment Instructions and Information
For this assignment, we will be using Google Colab. If you are new to Google Colab, you can find a brief introduction at the following link: [Google Colab Introduction](https://colab.research.google.com/notebooks/intro.ipynb).

**Important:** Before you start working on this notebook, make sure to save a copy to your own Google Drive. To do this, go to `File` -> `Save a copy in Drive`.

If you do not save a copy, you will not be able to save any changes you make.


### Submitting your assignment
Once you have completed the assignment, please submit your work as a `.ipynb` file on eClass in the "Assignment 7" section. To download your Colab notebook in the required format, follow these steps:

1. Save your notebook to ensure all changes are preserved.
2. Navigate to `File` -> `Download` -> `Download .ipynb`.

Make sure to save your notebook before downloading it!

### Questions and Autograding

Each section contains questions for you to solve, marked with the subsection heading "Question X.Y" (ex: the first problem is "Question 2.1").

For each question your solution must go within the following designated code block:

```python
### YOUR CODE HERE ###

######################
```

All questions will be autograded using [Otter-Grader](https://otter-grader.readthedocs.io/en/latest/).
The first two code cells in this notebook install the Otter-Grader package and download the test cases.
You should run these cells, otherwise the autograder will not work.

At the end of each question there is code that runs the autograder. For example, in Question 2.1 the code `grader.check("q2_1")` runs the autograder.
If you pass all the test cases for a question (ex: Question 2.1), you will see the following output:

**q2_1** passed!

If you do not pass all the test cases for a question, you will see which test cases you did not pass along with their corresponding error messages.

There are both public and private test cases. You only have access to the public test cases. This means that if you pass all the test cases in this notebook, you have passed all the public test cases.

After you submit the assignment, we will also run the private test cases. The public test cases account for 50% of the total mark, while the private test cases make up the remaining 50%. Therefore, if you pass all the test cases in the notebook, you are guaranteed a mark of at least 50%.

After each question description, the number of points (marks) the question is worth is indicated.
This is the total number of points for both the public and private test cases.
For each question, the public test cases are worth 50% of the points.


## Introduction

Welcome to Assignment 7! In this assignment, you will work on a binary and multiclass classification problem.
We will be focusing on the methods discussed in Chapter 9 of the [course notes](https://vladtkachuk4.github.io/machinelearning1/notes.pdf).

This assignment will guide you through the following sections:

1. [Understanding the Dataset](#part-1-understanding-the-dataset)
1. [Binary Classification](#part-2-binary-classification)
1. [Multiclass Classification](#part-3-multiclass-classification)

You can quickly navigate to each section by opening the table of contents on the left side of the screen.

Let's get started!


# Part 1: Understanding the Dataset

In this assignment, we will use the **UCI Wine dataset**, which is a classic dataset often used for classification tasks.
It contains information about various chemical properties of wines derived from three different cultivars in the same region in Italy.
The features include alcohol content, malic acid, ash, alcalinity of ash, magnesium, total phenols, flavanoids, nonflavanoid phenols, proanthocyanins, color intensity, hue, OD280/OD315 of diluted wines, and proline,
with the labels being the cultivar of the wine (Barolo (0), Grignolino (1), Barbera (2)).
The dataset contains 178 data points.

The dataset was introduced by Forina et al. (1991) in their paper on chemical analysis.

Let's begin by loading the dataset and taking a closer look at what it contains.

### Load the Dataset

First, lets import all the necessary libraries for this assignment.

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interactive_output
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.preprocessing import PolynomialFeatures
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

# Set a fixed random seed for reproducibility
random_seed = 42

We will use `load_wine` from the `sklearn.datasets` module to load the dataset.
Let's load the dataset and examine the features for the first few datapoints.
You should see that the dataset contains 13 features and a label (class).

In [ ]:
# Load the Wine dataset
wine = load_wine()

# Convert to a DataFrame
df = pd.DataFrame(wine.data, columns=wine.feature_names)
df['Class'] = wine.target

# Display some random rows of the dataset
df.sample(n=5, random_state=random_seed)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,Class
19,13.64,3.10,2.56,15.2,116.0,2.70,3.03,0.17,1.66,5.10,0.96,3.36,845.0,0
45,14.21,4.04,2.44,18.9,111.0,2.85,2.65,0.30,1.25,5.24,0.87,3.33,1080.0,0
140,12.93,2.81,2.70,21.0,96.0,1.54,0.50,0.53,0.75,4.60,0.77,2.31,600.0,2
30,13.73,1.50,2.70,22.5,101.0,3.00,3.25,0.29,2.38,5.70,1.19,2.71,1285.0,0
67,12.37,1.17,1.92,19.6,78.0,2.11,2.00,0.27,1.04,4.68,1.12,3.48,510.0,1


Now that we have loaded the dataset, we need to prepare it for use in our classification learners.

For better visualization of decision boundaries, we will only use 2 features in this assignment: alcohol and malic acid. We will store these features in a matrix $\mathbf{X}$ where each row represents a data point and each column represents a feature.
Additionally, we will store the labels (class) in a vector $\mathbf{Y}$.

For multiclass classification, we will use all the labels.
However, for binary classification, we will treat the first class as representing Barolo and group the other two classes together to represent Non-Barolo. We will call these labels $\mathbf{Y}_{\text{bin}}$.

In [ ]:
# Select only the alcohol and malic acid features for simplicity
X = df[['alcohol', 'malic_acid']].values
Y = df['Class'].values

# For binary classification, select class 0 (Barolo) and group the other two classes together to represent 1 (Not-Barolo)
binary_mask = Y == 0
Y_bin = np.where(binary_mask, 0, 1)

Next, we will split the data into training and testing sets. We will use 65% of the data for training and 35% for testing.

In [ ]:
# Split the data into training and testing sets
test_size = 0.35
X_train_bin, X_test_bin, Y_train_bin, Y_test_bin = train_test_split(X, Y_bin, test_size=test_size, random_state=random_seed)
X_train_mul, X_test_mul, Y_train_mul, Y_test_mul = train_test_split(X, Y, test_size=test_size, random_state=random_seed)

For the final step of preprocessing, we will normalize the data and add a column of ones to the feature matrix to account for the bias term. For an explanation of the normalization step see Assignment 4.

In [ ]:
# Normalize the features
X_mean_bin = X_train_bin.mean(axis=0)
X_std_bin = X_train_bin.std(axis=0) + 1e-8
X_train_norm_bin = (X_train_bin - X_mean_bin) / X_std_bin
X_test_norm_bin = (X_test_bin - X_mean_bin) / X_std_bin

X_mean_mul = X_train_mul.mean(axis=0)
X_std_mul = X_train_mul.std(axis=0) + 1e-8
X_train_norm_mul = (X_train_mul - X_mean_mul) / X_std_mul
X_test_norm_mul = (X_test_bin - X_mean_mul) / X_std_mul

# Append a column of 1s to the features for the bias term
X_train_norm_bin = np.hstack([np.ones((X_train_norm_bin.shape[0], 1)), X_train_norm_bin])
X_test_norm_bin = np.hstack([np.ones((X_test_norm_bin.shape[0], 1)), X_test_norm_bin])
X_train_norm_mul = np.hstack([np.ones((X_train_norm_mul.shape[0], 1)), X_train_norm_mul])
X_test_norm_mul = np.hstack([np.ones((X_test_norm_mul.shape[0], 1)), X_test_norm_mul])

### Plotting the Binary Dataset

The plot below visualizes the training and test sets for the binary classification dataset, which uses the Barolo and Not-Barolo classes.

In both plots, you should see two distinct clusters of points representing the two classes. Each cluster corresponds to one of the classes (Barolo or Not-Barolo), and the separation between the clusters indicates the differences in the features of the two classes.

In [ ]:
# @title Plot

# Function to plot the binary dataset without jitter
def plot_binary_dataset(show_plot):
    if show_plot:
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))

        # Plot training data
        axes[0].scatter(X_train_norm_bin[Y_train_bin == 0][:, 1], X_train_norm_bin[Y_train_bin == 0][:, 2], color='red', label='Barolo', s=50)
        axes[0].scatter(X_train_norm_bin[Y_train_bin == 1][:, 1], X_train_norm_bin[Y_train_bin == 1][:, 2], color='blue', label='Not-Barolo', s=50)
        axes[0].set_xlabel('Alcohol (normalized)')
        axes[0].set_ylabel('Malic Acid (normalized)')
        axes[0].set_title('Training Set')
        axes[0].set_ylim(-2.2, 3.8)
        axes[0].set_xlim(-2.8, 2.8)
        axes[0].legend()

        # Plot testing data
        axes[1].scatter(X_test_norm_bin[Y_test_bin == 0][:, 1], X_test_norm_bin[Y_test_bin == 0][:, 2], color='red', label='Barolo', s=50)
        axes[1].scatter(X_test_norm_bin[Y_test_bin == 1][:, 1], X_test_norm_bin[Y_test_bin == 1][:, 2], color='blue', label='Not-Barolo', s=50)
        axes[1].set_xlabel('Alcohol (normalized)')
        axes[1].set_ylabel('Malic Acid (normalized)')
        axes[1].set_title('Test Set')
        axes[1].set_ylim(-2.2, 3.8)
        axes[1].set_xlim(-2.8, 2.8)
        axes[1].legend()

        plt.show()
    else:
        clear_output()

# Create a checkbox widget
show_plot_checkbox_binary_dataset = widgets.Checkbox(value=False, description='Show Plot')

# Use interactive_output to link the function with the checkbox
interactive_plot_binary_dataset = interactive_output(plot_binary_dataset, {'show_plot': show_plot_checkbox_binary_dataset})

# Display the checkbox and the plot
display(show_plot_checkbox_binary_dataset, interactive_plot_binary_dataset)


Checkbox(value=False, description='Show Plot')

Output()

### Plotting the Multiclass Dataset

The plot below visualizes the training and test sets for the multiclass classification dataset, which includes the Barolo, Grignolino, and Barbera classes.

In both plots you should see three distinct clusters of points representing the three classes.

In [ ]:
# @title Plot

# Function to plot the training and testing sets for multiclass classification
def plot_multiclass_train_test(show_plot):
    if show_plot:
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))

        # Plot training data
        axes[0].scatter(X_train_norm_mul[Y_train_mul == 0][:, 1], X_train_norm_mul[Y_train_mul == 0][:, 2], color='red', label='Barolo', s=50)
        axes[0].scatter(X_train_norm_mul[Y_train_mul == 1][:, 1], X_train_norm_mul[Y_train_mul == 1][:, 2], color='blue', label='Grignolino', s=50)
        axes[0].scatter(X_train_norm_mul[Y_train_mul == 2][:, 1], X_train_norm_mul[Y_train_mul == 2][:, 2], color='green', label='Barbera', s=50)
        axes[0].set_xlabel('Alcohol (normalized)')
        axes[0].set_ylabel('Malic Acid (normalized)')
        axes[0].set_title('Training Set')
        axes[0].set_ylim(-2.2, 3.8)
        axes[0].set_xlim(-2.8, 2.8)
        axes[0].legend()

        # Plot testing data
        axes[1].scatter(X_test_norm_mul[Y_test_mul == 0][:, 1], X_test_norm_mul[Y_test_mul == 0][:, 2], color='red', label='Barolo', s=50)
        axes[1].scatter(X_test_norm_mul[Y_test_mul == 1][:, 1], X_test_norm_mul[Y_test_mul == 1][:, 2], color='blue', label='Grignolino', s=50)
        axes[1].scatter(X_test_norm_mul[Y_test_mul == 2][:, 1], X_test_norm_mul[Y_test_mul == 2][:, 2], color='green', label='Barbera', s=50)
        axes[1].set_xlabel('Alcohol (normalized)')
        axes[1].set_ylabel('Malic Acid (normalized)')
        axes[1].set_title('Test Set')
        axes[1].set_ylim(-2.2, 3.8)
        axes[1].set_xlim(-2.8, 2.8)
        axes[1].legend()

        plt.show()
    else:
        clear_output()

# Create a checkbox widget
show_plot_checkbox_multiclass_train_test = widgets.Checkbox(value=False, description='Show Plot')

# Use interactive_output to link the function with the checkbox
interactive_plot_multiclass_train_test = interactive_output(plot_multiclass_train_test, {'show_plot': show_plot_checkbox_multiclass_train_test})

# Display the checkbox and the plot
display(show_plot_checkbox_multiclass_train_test, interactive_plot_multiclass_train_test)

Checkbox(value=False, description='Show Plot')

Output()

# Part 2: Binary Classification

In this section we are interested in defining a learner for the binary classification problem.
As discussed in [Part 1](#part-1-understanding-the-dataset), we will treat the first class as Barolo (represented as the label $0$) and group the other two classes together to represent Non-Barolo (represented as the label $1$).
Thus, the set of labels is $\mathcal{Y} = \{0, 1\}$.
Our goal will be to implement the binary classification learner we learned in class.

We will first need to implement a logistic regression learner that uses batch gradient descent to optimize the weights.
To do this, lets implement the sigmoid (or logistic) function used as the predictor function by the logistic regression learner

### Question 2.1

In class we learned that the sigmoid function is defined as:
$$\sigma(z) = \frac{1}{1 + e^{-z}}$$
However, if $z$ is a large negative number, $e^{-z}$ will be a very large positive number and can cause an overflow error.
To avoid this, we can rewrite the sigmoid function as:
$$\sigma(z) = \frac{1}{1 + e^{-z}} = \frac{1}{1 + e^{-z}} \cdot \frac{e^z}{e^z} = \frac{e^z}{e^z + 1}$$
Notice that this form of the sigmoid function will not overflow for large negative values of $z$; however, it may overflow for large positive values of $z$.

Thus, the solution is to check if the value of $z$ is negative or positive and use the appropriate form of the sigmoid function.
In particular, if $z$ is positive you should use the first form of the sigmoid function, and if $z$ is negative you should use the second form of the sigmoid function.

For your implementation of the sigmoid function you should assume the input is a vector represented as a numpy arrary.
You should then return a vector of the same shape where each element is the sigmoid function applied to the corresponding element of the input vector.
Using vectors as input and output is often preferred in practice since it allows for a more efficient implementation.
In particular, you can avoid for loops in your implementation by noting that `numpy` will apply its functions elementwise to numpy arrays (ex: `np.exp(Z)` will apply the exponential function to each element of the array `Z`).
Similarly, arithmetic operations are also applied elementwise (ex: `1 + Z` will add 1 to each element of the array `Z`).
It turns out that this is much faster than using for loops in Python.

We have already started the implementation by initializing the output array `result` to an array (vector) of the same shape as the input `Z`.

Complete the implemention of `sigmoid`.

_Points:_ 8

In [ ]:
def sigmoid(Z):
    '''
    Compute the sigmoid function for the input array Z.
    This implementation uses conditional logic to avoid overflow issues for large positive and negative values of Z.

    Parameters:
    Z (numpy array): Input array of shape (n,) for which to compute the sigmoid function.

    Returns:
    result (numpy array): An array with the same shape as Z, containing the sigmoid function values for each element of Z.
    '''
    result = np.zeros(Z.shape)  # Initialize result array with the same shape as Z

    ### YOUR CODE HERE ###
    positive_mask = Z >= 0
    negative_mask = Z < 0

    result[positive_mask] = 1 / (1 + np.exp(-Z[positive_mask]))

    result[negative_mask] = np.exp(Z[negative_mask]) / (1 + np.exp(Z[negative_mask]))


    ######################

    return result

In [ ]:
grader.check("q2_1")

q2_1 results: All test cases passed!

### Question 2.2

Now we are ready to implement the logistic regression learner using batch gradient descent.
That is, you need to finish the implementation of the `bgd_logistic_regression_learner` function by adding the batch gradient descent update rule with a constant step size $\eta^{(t)} = \eta$.
Recall that the update rule for batch gradient descent is:
$$\mathbf{w}^{(t+1)} = \mathbf{w}^{(t)} - \eta^{(t)} \nabla \hat L(\mathbf{w}^{(t)}) \quad \text{where} \quad \hat L(\mathbf{w}) = \frac{1}{n} \sum_{i=1}^{n} \ell(\sigma(\mathbf{x}_i^\top \mathbf{w}), y_i),$$
and $\ell$ is the binary cross-entropy loss function:
$$\ell(\hat y, y) = -y \log(\hat y) - (1 - y) \log(1 - \hat y).$$

Complete the implementation of `bgd_logistic_regression_learner`.

_Points:_ 8

In [ ]:
def bgd_logistic_regression_learner(X, Y, step_size=0.01, epochs=10, random_seed=42):
    '''
    Solves logistic regression using batch gradient descent.

    Parameters:
    X (numpy array): Feature matrix of size (n, d+1), where n is the number of samples
                     and d is the number of features. The first column should be all 1s.
    Y (numpy array): Target vector of size (n,).
    step_size (float): The step size for gradient descent.
    epochs (int): The number of iterations to run gradient descent.
    random_seed (int, optional): The seed for the random number generator.

    Returns:
    predictor (function): A function that takes a feature vector or matrix and returns a predicted value.
    w (numpy array): The final weights after applying gradient descent for the specified epochs.
    '''
    n, d = X[:,1:].shape
    np.random.seed(random_seed)
    w = np.random.randn(d+1) # initialize the weights randomly

    ### YOUR CODE HERE ###

    for epoch in range(epochs):

        preds = sigmoid(X @ w)
        gradient = (1 / n) * (X.T @ (preds - Y))
        w = w - step_size * gradient


    ######################

    def predictor(x):
        return sigmoid(x @ w)

    return predictor, w


In [ ]:
grader.check("q2_2")

q2_2 results: All test cases passed!

### Question 2.3

To make sure the implementation of `bgd_logistic_regression_learner` is correct we would like to plot the estimated loss of the predictor output by the learner as a function of the number of epochs.
To do this you will need to implement the `estimated_loss`.
The estimated loss of a predictor $f$, and loss function $\ell$ is defined as:
$$\hat L(f) = \frac{1}{n} \sum_{i=1}^{n} \ell(f(\mathbf{x}_i), y_i).$$

You can assume that the predictor `f` and loss function `loss_function` are implemented as functions that take a vector of inputs and return a vector of outputs.
You may find the function `np.mean` useful for this implementation.

Complete the implemention of `estimated_loss`.

_Points:_ 4

In [ ]:
def estimated_loss(f, X, Y, loss_function):
    """
    Compute the estimated loss using the input loss function for a given predictor over a dataset.

    Parameters:
    f (function): A function that takes a feature vector or matrix and returns a predicted value.
    X (numpy array): Feature matrix of size (n, d+1), where n is the number of samples
                     and d is the number of features. The first column should be all 1s.
    Y (numpy array): Target vector of size (n,).
    loss_function (function): A function that takes true labels and predicted values and returns the loss.

    Returns:
    loss (float): The estimated loss.
    """

    ### YOUR CODE HERE ###


    predicted_values = f(X)
    loss = np.mean(loss_function(predicted_values, Y))



    ######################

    return loss


In [ ]:
grader.check("q2_3")

q2_3 results: All test cases passed!

### Question 2.4

To plot the `estimated_loss` you implemented in [Question 2.3](#question-2.3), we need to pick a specific loss function.
We mentioned in [Question 2.2](#question-2.2) that the `bgd_logistic_regression_learner` is using the binary cross-entropy loss function.
Thus, you should implement the binary cross-entropy loss function, which is defined as:
$$\ell(\hat y, y) = -y \log(\hat y) - (1 - y) \log(1 - \hat y).$$

Similar to the `sigmoid` function, you should assume the input is a vector represented as a numpy arrary.
You should then return a vector of the same shape where each element is the binary cross-entropy loss applied to the corresponding elements of the input vectors.
You may find the function `np.log` useful for this implementation.

Complete the implemention of `binary_cross_entropy_loss`.

_Points:_ 4

In [ ]:
def binary_cross_entropy_loss(Y_pred, Y):
    """
    Compute the cross-entropy loss for binary classification.

    Parameters:
    Y_pred (numpy array): Predicted probabilities of shape (n,).
    Y (numpy array): True labels (0 or 1) of shape (n,).

    Returns:
    loss (numpy array): Cross-entropy loss of shape (n,).
    """

    # Clip predictions to avoid log(0)
    Y_pred = np.clip(Y_pred, 1e-15, 1 - 1e-15)

    ### YOUR CODE HERE ###

    loss = -Y * np.log(Y_pred) - (1-Y) * np.log(1-Y_pred)

    ######################

    return loss

In [ ]:
grader.check("q2_4")

q2_4 results: All test cases passed!

### Plotting the Estimated Cross-Entropy Loss

Below is a plot of the estimated loss (using the cross-entropy loss function) for the predictor output by the `bgd_logistic_regression_learner` as a function of the number of epochs.
If your implementation of `bgd_logistic_regression_learner` is correct, you should see the loss decrease as the number of epochs increases.

In [ ]:
# @title Plot

# Function to plot the loss values
def plot_ce_loss_binary(show_plot):
    # Define the number of epochs
    epochs = 200

    # Initialize lists to store the loss values
    loss_values = []

    # Train the model and compute the loss for each epoch
    for epoch in range(1, epochs + 1):
        predictor, w = bgd_logistic_regression_learner(X_train_norm_bin, Y_train_bin, step_size=0.2, epochs=epoch, random_seed=random_seed)
        loss = estimated_loss(predictor, X_train_norm_bin, Y_train_bin, binary_cross_entropy_loss)
        loss_values.append(loss)
    if show_plot:
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, epochs + 1), loss_values, linewidth=3)  # Set linewidth to 2
        plt.xlabel('Epochs')
        plt.ylabel('Estimated Cross-Entropy Loss')
        plt.grid(True)
        plt.show()
    else:
        clear_output()

# Create a checkbox widget
show_plot_checkbox_ce_loss_binary = widgets.Checkbox(value=False, description='Show Plot')

# Use interactive_output to link the function with the checkbox
interactive_plot_ce_loss_binary = interactive_output(plot_ce_loss_binary, {'show_plot': show_plot_checkbox_ce_loss_binary})

# Display the checkbox and the plot
display(show_plot_checkbox_ce_loss_binary, interactive_plot_ce_loss_binary)

Checkbox(value=False, description='Show Plot')

Output()

### Binary Classification Learner

Recall that our objective in binary classification is to predict the class of a data point as either Barolo (0) or Not-Barolo (1).
So far we have implemented the logistic regression learner, which returns a predictor that outputs a proability of the data point being Not-Barolo.
To make a prediction, we need to convert this probability to a class label.
We can do this by thresholding the probability at 0.5, such that if the probability is greater than 0.5 we predict Not-Barolo, otherwise we predict Barolo.
This is implemented for you as `binary_classification_learner` below.

In [ ]:
def binary_classification_learner(X, Y, step_size=0.01, epochs=10, random_seed=42):
    """
    Trains a binary classification model using batch gradient descent and returns a predictor function.

    Parameters:
    X (numpy array): Feature matrix of size (n, d+1), where n is the number of samples
                     and d is the number of features. The first column should be all 1s.
    Y (numpy array): Target vector of size (n,).
    step_size (float): The step size for gradient descent.
    epochs (int): The number of iterations to run gradient descent.
    decision_boundary (float): The decision boundary for classification.
    random_seed (int, optional): The seed for the random number generator.

    Returns:
    predictor (function): A function that takes a feature vector or matrix and returns a predicted class (0 or 1).
    w (numpy array): The final weights after applying gradient descent for the specified epochs.
    """
    # Use the bgd_logistic_regression_learner to train the model
    predictor, w = bgd_logistic_regression_learner(X, Y, step_size, epochs, random_seed)

    # Define a new predictor function that uses the decision boundary
    def binary_predictor(x):
        return (predictor(x) >= 0.5).astype(int)

    return binary_predictor, w

### Question 2.5

To evaluate the performance of the binary classification learner, we should use the zero-one loss function, since it measures the number of misclassifications.
This is in contrast to the cross-entropy loss function, which measures the difference between the predicted probability and the true label.
Since our objective is to predict class labels correctly, the zero-one loss is more appropriate for evaluating the performance of a binary classification learner.

Recall however that the `binary_classification_learner` makes use of the `bgd_logistic_regression_learner`, which is actually minimizing the cross-entropy loss instead of the zero-one loss.
Intuitively, we might expect that if we predict the probability of a data point being Not-Barolo correctly, we should also predict the class label correctly.
To confirm this intuition, we would like to plot the estimated loss of the predictor output by the `binary_classification_learner` as a function of the number of epochs using the zero-one loss function.

To do this you first need to implement the zero-one loss function, which is defined as:
$$\ell(\hat y, y) = \begin{cases} 0 & \text{if } \hat y = y \\ 1 & \text{if } \hat y \neq y \end{cases}$$

As before, you should assume the inputs are vectors represented as numpy arrarys.

Complete the implemention of `zero_one_loss`.


_Points:_ 4

In [ ]:
def zero_one_loss(Y_pred, Y):
    """
    Compute the 0-1 loss for binary classification.

    Parameters:
    Y_pred (numpy array): Predicted labels of shape (n,).
    Y (numpy array): True labels of shape (n,).

    Returns:
    loss (numpy array): 0-1 loss of shape (n,).
    """

    ### YOUR CODE HERE ###

    loss = (Y_pred != Y).astype(int)


    ######################

    return loss

In [ ]:
grader.check("q2_5")

q2_5 results: All test cases passed!

### Plotting the Estimated Zero-One Loss

Below is a plot of the estimated loss (using the zero-one loss function) for the predictor output by the `binary_classification_learner` as a function of the number of epochs.
We also plot the estimated loss using the cross-entropy loss function for comparison.

If your implementation of `zero_one_loss` is correct, you should see the zero-one loss decrease as the number of epochs increases.
However, it likely will not decrease as smoothly as the cross-entropy loss, since the `bgd_logistic_regression_learner` is not directly minimizing the zero-one loss.

In [ ]:
# @title Plot

# Function to plot the loss values
def plot_ce_zo_loss_bin(show_plot):
    # Define the number of epochs
    epochs = 200

    # Initialize lists to store the loss values
    binary_cross_entropy_loss_values = []
    zero_one_loss_values = []

    # Train the model and compute the loss for each epoch
    for epoch in range(1, epochs + 1):
        predictor_log, w = bgd_logistic_regression_learner(X_train_norm_bin, Y_train_bin, step_size=0.2, epochs=epoch, random_seed=random_seed)
        predictor_bin, w = binary_classification_learner(X_train_norm_bin, Y_train_bin, step_size=0.2, epochs=epoch, random_seed=random_seed)
        binary_cross_entropy_loss_value = estimated_loss(predictor_log, X_train_norm_bin, Y_train_bin, binary_cross_entropy_loss)
        zero_one_loss_value = estimated_loss(predictor_bin, X_train_norm_bin, Y_train_bin, zero_one_loss)
        binary_cross_entropy_loss_values.append(binary_cross_entropy_loss_value)
        zero_one_loss_values.append(zero_one_loss_value)

    if show_plot:
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, epochs + 1), binary_cross_entropy_loss_values, label='Cross-Entropy Loss', linewidth=3)
        plt.plot(range(1, epochs + 1), zero_one_loss_values, label='Zero-One Loss', linewidth=3)
        plt.xlabel('Epochs')
        plt.ylabel('Estimated Loss')
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        clear_output()

# Create a checkbox widget
show_plot_checkbox_ce_zo_loss_binary = widgets.Checkbox(value=False, description='Show Plot')

# Use interactive_output to link the function with the checkbox
interactive_plot_loss_ce_zo_loss_binary = interactive_output(plot_ce_zo_loss_bin, {'show_plot': show_plot_checkbox_ce_zo_loss_binary})

# Display the checkbox and the plot
display(show_plot_checkbox_ce_zo_loss_binary, interactive_plot_loss_ce_zo_loss_binary)

Checkbox(value=False, description='Show Plot')

Output()

### Plotting a Linear Decision Boundary

The binary classification predictor outputs the class 1 if the logistic regression predictor outputs a probability greater than 0.5.
Since the logistic regression predictor is given by $\sigma(\mathbf{x}^\top \mathbf{w})$, it implies that $\mathbf{x}^\top \mathbf{w} \ge 0$ when $\sigma(\mathbf{x}^\top \mathbf{w}) \ge 0.5$.
Thus, to visualize when the binary classification predictor decides to output the class 0, or 1, we can plot the line $\mathbf{x}^\top \mathbf{w} = 0$, which is called the *decision boundary*.

The decision boundary is learned based on the training dataset shown on the left.
We plot the same decision boundary on the right to show how well it does on the test set.

In [ ]:
# @title Plot

# Function to show/hide the plot
def plot_db_binary(show_plot):
    # Define the decision boundary function
    def plot_decision_boundary(X, Y, f, title, ax):
        # Create a mesh grid
        x_min, x_max = -2.8, 2.8  # Hardcoded x-limits
        y_min, y_max = -2.2, 3.8  # Hardcoded y-limits
        xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                            np.arange(y_min, y_max, 0.01))

        # Compute the decision boundary
        Z = f(np.c_[np.ones((xx.ravel().shape[0], 1)), xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)

        # Plot the decision boundary
        ax.contourf(xx, yy, Z, alpha=0.8, levels=[0, 0.5, 1], colors=['#FFAAAA', '#AAAAFF'])
        ax.scatter(X[Y == 0][:, 1], X[Y == 0][:, 2], color='red', label='Barolo', s=50)
        ax.scatter(X[Y == 1][:, 1], X[Y == 1][:, 2], color='blue', label='Not-Barolo', s=50)
        ax.set_xlabel('Alcohol (normalized)')
        ax.set_ylabel('Malic Acid (normalized)')
        ax.set_title(title)

        # Hardcoded limits to ensure consistency across both plots
        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)
        ax.legend()

    epochs = 10000

    if show_plot:
        # Create subplots
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))

        predictor, w = binary_classification_learner(X_train_norm_bin, Y_train_bin, step_size=0.2, epochs=epochs, random_seed=random_seed)
        # Plot the decision boundary on the training set
        plot_decision_boundary(X_train_norm_bin, Y_train_bin, predictor, 'Training Set', axes[0])

        # Plot the decision boundary on the test set
        plot_decision_boundary(X_test_norm_bin, Y_test_bin, predictor, 'Test Set', axes[1])

        plt.show()
    else:
        clear_output()

# Create a checkbox widget
show_plot_checkbox_db_binary = widgets.Checkbox(value=False, description='Show Plot')

# Use interactive_output to link the function with the checkbox
interactive_plot_db_binary = interactive_output(plot_db_binary, {'show_plot': show_plot_checkbox_db_binary})

# Display the checkbox and the plot
display(show_plot_checkbox_db_binary, interactive_plot_db_binary)

Checkbox(value=False, description='Show Plot')

Output()

From the above plot we might notice that if the decision boundary did not have to be a straight line, we could have achieved a better separation between the two classes.
To achieve this, we can change the function class used by the logistic regression learner.
So far the logistic regression learner has been using the following logistic function class:
$$\bar{\mathcal{F}}_1 = \{ f | f: \mathbb{R}^{d+1} \to \{0, 1\}, \text{ where } f(\mathbf{x}) = \sigma(\mathbf{x}^\top \mathbf{w}), \text { and } \mathbf{w} \in \mathbb{R}^{d+1}\}.$$
The reason the decision boundary was a straight line is because the input to the sigmoid function was a linear function of the features.
To allow for more complex decision boundaries, we can change the features to be polynomial features of the original features.
For a degree $p$, the polynomial logistic function class is defined as:
$$\bar{\mathcal{F}_p} = \{ f | f: \mathbb{R}^{d+1} \to \{0, 1\}, \text{ where } f(\mathbf{x}) = \sigma(\phi_p(\mathbf{x})^\top \mathbf{w}), \text { and } \mathbf{w} \in \mathbb{R}^{\bar{p}+1}\}.$$
where $\phi_p$ is the polynomial feature function that takes a feature vector $\mathbf{x}$ and a degree $p$ and returns a feature vector of all polynomial features up to degree $p$.

The function `phi_p` below is an implementation of the polynomial feature function $\phi_p$ for a feature matrix `X` and a degree `p`.


In [ ]:
def phi_p(X, p):
    """
    Transforms the input feature matrix X into polynomial features of degree p and normalizes them.

    Parameters:
    X (numpy.ndarray): Input feature matrix where each row is a feature vector.
    p (int): Degree of the polynomial features.

    Returns:
    poly_features (numpy.ndarray): Transformed and normalized polynomial features.
    mean (numpy.ndarray): Mean of the polynomial features used for normalization.
    std (numpy.ndarray): Standard deviation of the polynomial features used for normalization.
    """
    # Create PolynomialFeatures instance with the desired degree
    poly = PolynomialFeatures(degree=p, include_bias=False)

    # Transform the feature matrix to polynomial features
    poly_features = poly.fit_transform(X)

    return poly_features

### Plotting Polynomial Decision Boundaries

Below is a plot of the decision boundary for different polynomial degrees $p$.
You should see that as $p$ increases the decision boundary becomes more complex and can better separate the two classes in the training set.
However, as $p$ increases the decision boundary may become too complex and overfit the training data, leading to poor performance on the test set.

In [ ]:
# @title Plot

# Function to show/hide the plot and update the polynomial degree
def update_plot_db_binary_poly(show_plot, degree):

    # Define the number of epochs and polynomial degree
    epochs = 10000

    # Define the decision boundary function for polynomial features
    def plot_decision_boundary_poly(X, Y, f, degree, title, X_poly_mean=None, X_poly_std=None, ax=None):
        # Create a mesh grid
        x_min, x_max = -2.8, 2.8  # Hardcoded x-limits
        y_min, y_max = -2.2, 3.8  # Hardcoded y-limits
        xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                            np.arange(y_min, y_max, 0.01))

        # Transform the mesh grid to polynomial features
        poly_features = phi_p(np.c_[xx.ravel(), yy.ravel()], degree)

        # Normalize the polynomial features using the same mean and std as the training set
        poly_features = (poly_features - X_poly_mean) / X_poly_std

        # Append a column of 1s to the features for the bias term
        poly_features = np.hstack([np.ones((poly_features.shape[0], 1)), poly_features])

        # Compute the decision boundary
        Z = f(poly_features)
        Z = Z.reshape(xx.shape)

        # Plot the decision boundary
        ax.contourf(xx, yy, Z, alpha=0.8, levels=[0, 0.5, 1], colors=['#FFAAAA', '#AAAAFF'])
        ax.scatter(X[Y == 0][:, 1], X[Y == 0][:, 2], color='red', label='Barolo', s=50)
        ax.scatter(X[Y == 1][:, 1], X[Y == 1][:, 2], color='blue', label='Not-Barolo', s=50)
        ax.set_xlabel('Alcohol (normalized)')
        ax.set_ylabel('Malic Acid (normalized)')
        ax.set_ylim(y_min, y_max)
        ax.set_xlim(x_min, x_max)
        ax.set_title(title)
        ax.legend()

    if show_plot:
        # Transform the input features to polynomial features of the selected degree
        X_poly_train = phi_p(X_train_norm_bin[:, 1:], degree)
        X_poly_test = phi_p(X_test_norm_bin[:, 1:], degree)

        # Normalize the polynomial features
        X_poly_mean = X_poly_train.mean(axis=0)
        X_poly_std = X_poly_train.std(axis=0) + 1e-8
        X_poly_train = (X_poly_train - X_poly_mean) / X_poly_std
        X_poly_test = (X_poly_test - X_poly_mean) / X_poly_std

        # Append a column of 1s to the features for the bias term
        X_poly_train = np.hstack([np.ones((X_poly_train.shape[0], 1)), X_poly_train])
        X_poly_test = np.hstack([np.ones((X_poly_test.shape[0], 1)), X_poly_test])

        # Train the model using polynomial features
        predictor_poly, w_poly = binary_classification_learner(X_poly_train, Y_train_bin, step_size=0.2, epochs=epochs, random_seed=random_seed)

        # Create subplots
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))

        # Plot the decision boundary on the training set
        plot_decision_boundary_poly(X_poly_train, Y_train_bin, predictor_poly, degree, f'Training Set (Polynomial Degree {degree})', X_poly_mean, X_poly_std, ax=axes[0])

        # Plot the decision boundary on the test set
        plot_decision_boundary_poly(X_poly_test, Y_test_bin, predictor_poly, degree, f'Test Set (Polynomial Degree {degree})', X_poly_mean, X_poly_std, ax=axes[1])

        plt.show()
    else:
        clear_output()

# Create a checkbox widget
show_plot_checkbox_db_binary_poly = widgets.Checkbox(value=False, description='Show Plot')

# Create a slider widget for selecting the polynomial degree
degree_slider_db_binary_poly = widgets.IntSlider(value=1, min=1, max=10, step=0.5, description='Degree')

# Use interactive_output to link the function with the checkbox and slider
interactive_plot_db_binary_poly = interactive_output(update_plot_db_binary_poly, {'show_plot': show_plot_checkbox_db_binary_poly, 'degree': degree_slider_db_binary_poly})

# Display the checkbox, slider, and the plot
display(show_plot_checkbox_db_binary_poly, degree_slider_db_binary_poly, interactive_plot_db_binary_poly)

Checkbox(value=False, description='Show Plot')

IntSlider(value=1, description='Degree', max=10, min=1, step=0)

Output()

To see which polynomial degree $p$ performs best on the test set, we will plot the estimated zero-one loss of the predictor output by the `binary_classification_learner` as a function of the polynomial degree $p$.
We also plot the estimated zero-one loss on the training set for comparison.

You should see that on the training set the zero-one loss decreases as $p$ increases, since the decision boundary can better separate the two classes.
You should also see that on the test set the zero-one loss decreases as $p$ increases up to a certain point, after which the zero-one loss increases.
If we were trying to select the best predictor, we would select the predictor with the lowest zero-one loss on the test set, which should be at $p=4$ in the plot below.

In [ ]:
# @title Plot

# Function to show/hide the plot
def plot_loss_vs_deg_bin(show_plot):
    epochs = 10000
    # Define the range of polynomial degrees
    degrees_binary = range(1, 11)

    # Initialize lists to store the loss values
    train_zero_one_loss_values_by_degree = []
    test_zero_one_loss_values_by_degree = []

    # Train the model and compute the loss for each polynomial degree
    for degree in degrees_binary:
        # Transform the input features to polynomial features of the selected degree
        X_poly_train = phi_p(X_train_norm_bin[:, 1:], degree)
        X_poly_test = phi_p(X_test_norm_bin[:, 1:], degree)

        # Normalize the polynomial features
        X_poly_mean = X_poly_train.mean(axis=0)
        X_poly_std = X_poly_train.std(axis=0) + 1e-8
        X_poly_train = (X_poly_train - X_poly_mean) / X_poly_std
        X_poly_test = (X_poly_test - X_poly_mean) / X_poly_std

        # Append a column of 1s to the features for the bias term
        X_poly_train = np.hstack([np.ones((X_poly_train.shape[0], 1)), X_poly_train])
        X_poly_test = np.hstack([np.ones((X_poly_test.shape[0], 1)), X_poly_test])

        # Train the model using polynomial features
        predictor_poly, w_poly = binary_classification_learner(X_poly_train, Y_train_bin, step_size=0.2, epochs=epochs, random_seed=random_seed)

        # Compute the train and test 0-1 loss
        train_zero_one_loss_value = estimated_loss(predictor_poly, X_poly_train, Y_train_bin, zero_one_loss)
        test_zero_one_loss_value = estimated_loss(predictor_poly, X_poly_test, Y_test_bin, zero_one_loss)

        # Store the loss values
        train_zero_one_loss_values_by_degree.append(train_zero_one_loss_value)
        test_zero_one_loss_values_by_degree.append(test_zero_one_loss_value)

    if show_plot:
        plt.figure(figsize=(10, 6))
        plt.plot(degrees_binary, train_zero_one_loss_values_by_degree, label='Train Zero-One Loss', linewidth=3)
        plt.plot(degrees_binary, test_zero_one_loss_values_by_degree, label='Test Zero-One Loss', linewidth=3)
        plt.xlabel('Polynomial Degree')
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        clear_output()

# Create a checkbox widget
show_plot_checkbox_loss_vs_deg_bin = widgets.Checkbox(value=False, description='Show Plot')

# Use interactive_output to link the function with the checkbox
interactive_plot_loss_vs_deg_bin = interactive_output(plot_loss_vs_deg_bin, {'show_plot': show_plot_checkbox_loss_vs_deg_bin})

# Display the checkbox and the plot
display(show_plot_checkbox_loss_vs_deg_bin, interactive_plot_loss_vs_deg_bin)

Checkbox(value=False, description='Show Plot')

Output()

# Part 3: Multiclass Classification

In multiclass classification, the set of labels $\mathcal{Y}$ contains more than 2 classes.
In our particular case, the set of labels $\mathcal{Y} = \{0, 1, 2\}$ represents the cultivar of the wine (Barolo, Grignolino, Barbera).
Thus, we will be using all the labels in our dataset from now on, represented as the matrix `Y`.

In this section our goal is to implement the multiclass classification learner we learned in class.

### Question 3.1

We will first need to implement the `softmax` function, which is used as the predictor function by the softmax regression learner.
In class we learned that the softmax function is defined as:
$$\sigma_y(\mathbf{z}) = \sigma_y(z_0, \dots, z_{\mathrm{K-1}}) = \frac{e^{z_y}}{\sum_{q=0}^{\mathrm{K}-1} e^{z_q}}$$
where $\mathbf{z} \in \mathbb{R}^{\mathrm{K}}$, $\mathrm{K}$ is the number of classes and $y \in \cal{Y}$.

If for some $y$, $z_y$ is large, then $e^{z_y}$ will be very large and can cause an overflow error.
To avoid this, we can rewrite the softmax function as:
$$\sigma_y(\mathbf{z}) = \sigma_y(z_0, \dots, z_{\mathrm{K}-1}) = \frac{e^{z_y}}{\sum_{q=0}^{\mathrm{K}-1} e^{z_q}} = \frac{e^{z_y}}{\sum_{q=0}^{\mathrm{K}-1} e^{z_q}} \cdot \frac{e^{-\max_{l} z_l}}{e^{-\max_{l} z_l}} = \frac{e^{z_y - \max_{l} z_l}}{\sum_{q=0}^{\mathrm{K}-1} e^{z_q - \max_{l} z_l}}.$$
This way the exponential function will never take as input a number larger than 0, which will prevent overflow errors.

For similar reasons as we have discussed in [Question 2.1](#question-2.1), it is often preferred to use vectors or matrices as input and output for the softmax function.
In particular, we can store all $\mathrm{K}$ different softmax outputs into a single vector and define a vector softmax function as:
$$\sigma(\mathbf{z}) = (\sigma_0(\mathbf{z}), \dots, \sigma_{\mathrm{K}-1}(\mathbf{z}))^\top \in [0, 1]^{\mathrm{K}}.$$
We would also like to compute the softmax function for multiple data points at once.
That means the input to the `softmax` function will be a matrix `Z` with $n$ rows and $\mathrm{K}$ columns, where each row represents a data point and each column represents a class.

In your implementation of the softmax function you should assume the input is a matrix represented as a numpy arrary.
You should then return a matrix of the same shape where each row is the vector softmax function $\sigma$ applied to the corresponding row of the input matrix.

You may find the functions: `np.exp`, `np.sum`, and `np.max` useful for this implementation.

Complete the implementation of `softmax`.

_Points:_ 8

In [ ]:
def softmax(Z):
    """
    Compute the softmax of each row of the input array.

    Parameters:
    z (numpy array): Input array of shape (n, K) where n is the number of samples and K is the number of classes.

    Returns:
    result (numpy array): Softmax probabilities of shape (n, K).
    """

    ### YOUR CODE HERE ###
    if Z.ndim == 1:
        Z = Z[:, np.newaxis]  # Reshape to (n, 1)

    Z_shifted = Z - np.max(Z, axis=1, keepdims=True)  # Prevent overflow
    exp_Z = np.exp(Z_shifted)
    sum_exp_Z = np.sum(exp_Z, axis=1, keepdims=True)
    result = exp_Z / sum_exp_Z

    ######################

    return result

In [ ]:
grader.check("q3_1")

q3_1 results: All test cases passed!

### Question 3.2

Now we are ready to implement the softmax regression learner using batch gradient descent.
That is, you need to finish the implementation of the `bgd_softmax_regression_learner` function by adding the batch gradient descent update rule with a constant step size $\eta^{(t)} = \eta$.
Recall that the update rule (for class $y \in \cal{Y} = \{0, \dots, \mathrm{K} - 1\}$) for batch gradient descent is:
$$\mathbf{w}_y^{(t+1)} = \mathbf{w}_y^{(t)} - \eta^{(t)} \nabla_{\mathbf{w}_y} \hat L(\mathbf{w}_0^{(t)}, \cdots, \mathbf{w}_{K-1}^{(t)}) \quad \text{where} \quad \nabla_{\mathbf{w}_y} \hat L(\mathbf{w}_0^{(t)}, \cdots, \mathbf{w}_{K-1}^{(t)}) = \left(\frac{\partial \hat L}{\partial \mathbf{w}_{y0}}, \dots, \frac{\partial \hat L}{\partial \mathbf{w}_{yd}}\right)^\top,$$
$$\hat L(\mathbf{w}_0^{(t)}, \cdots, \mathbf{w}_{K-1}^{(t)}) = \frac{1}{n} \sum_{i=1}^{n} \ell(\sigma(\mathbf{x}_i^\top \mathbf{w}_0, \dots, \mathbf{x}_i^\top \mathbf{w}_{K-1}), y_i),$$
The function $\textbf{I}_{\{y = q\}}(y)$ is the indicator function that is 1 if $y = q$ and 0 otherwise, and $\hat y_q$ is the $q$-th element of the vector $\hat y$.

Notice that $\hat y \in [0, 1]^{\mathrm{K}}$ is the output of the softmax function and $y \in \cal{Y}$ is the true label.
Importantly, $\cal{Y} \neq [0, 1]^{\mathrm{K}}$.
In your implementation of `bgd_softmax_regression_learner` you should assume that `Y` is a vector of labels from the set $\cal{Y}$.
To make the gradient calculation simpler, you may find it useful to convert `Y` into a matrix with $n$ rows and $\mathrm{K}$ columns by using the following mapping from an element $y \in \cal{Y}$ to an element $y' \in [0, 1]^\mathrm{K}$ (when $\mathrm{K} = 3$):
$$y' = \begin{cases} (1, 0, 0)^\top & \text{if } y = 0 \\ (0, 1, 0)^\top & \text{if } y = 1 \\ (0, 0, 1)^\top & \text{if } y = 2 \end{cases}.$$

Since there are $\mathrm{K}$ weight vectors $\mathbf{w}_0, \dots, \mathbf{w}_{\mathrm{K}-1}$, we will store them in a matrix $\mathbf{W}$ with $d+1$ rows and $\mathrm{K}$ columns, where each column represents the weights for a class.

Complete the implementation of `bgd_softmax_regression_learner`.

_Points:_ 12

In [ ]:
def bgd_softmax_regression_learner(X, Y, step_size=0.01, epochs=10, random_seed=42):
    """
    Trains a softmax regression model using batch gradient descent.

    Parameters:
    X (numpy array): Feature matrix of size (n, d+1), where n is the number of samples
                     and d is the number of features. The first column should be all 1s.
    Y (numpy array): Target vector of size (n,) with class labels.
    step_size (float): The step size for gradient descent.
    epochs (int): The number of iterations to run gradient descent.
    random_seed (int, optional): The seed for the random number generator.

    Returns:
    predictor (function): A function that takes a feature vector or matrix and returns predicted probabilities.
    W (numpy array): The final weights after applying gradient descent for the specified epochs.
    """
    np.random.seed(random_seed)
    n, d = X[:, 1:].shape
    K = len(np.unique(Y))  # Number of classes
    W = np.random.randn(d+1, K)  # Initialize the weights randomly

    ### YOUR CODE HERE ###

    Y_one_hot = np.zeros((n, K))
    for i in range(n):
        Y_one_hot[i, Y[i]] = 1

    for epoch in range(epochs):

      gradient = (X.T @ (softmax(X @ W) - Y_one_hot)) / n
      W -= step_size * gradient

    ######################

    def predictor(x):
        return softmax(x @ W)

    return predictor, W

In [ ]:
grader.check("q3_2")

q3_2 results: All test cases passed!

### Question 3.3

We would like to plot the estimated loss of the predictor output by the `bgd_softmax_regression_learner` as a function of the number of epochs.
To do this we need to implement the multiclass cross-entropy loss function, which we will use in the `estimated_loss` function.

As discussed in [Question 3.2](#question-3.2), the multiclass cross-entropy loss function is defined as:
$$\ell(\hat y, y) = -\sum_{q=0}^{\mathrm{K}-1} \textbf{I}_{\{y = q\}}(y) \log(\hat y_q) \quad \text{where} \quad \hat y \in [0, 1]^{\mathrm{K}}, y \in \cal{Y}.$$

Similar to the previous question you should assume that `Y` contains labels from the set $\cal{Y}$.
However, since `Y_hat` contains outputs of the softmax function, the rows of `Y_hat` will be elements of the set $[0, 1]^K$.
You may again find it useful to convert `Y` into a matrix of labels before using it in the loss calculation.
The reason for this is that the loss function can be written as:
$$\ell(\hat y, \bar{y}) = -\sum_{q=0}^{\mathrm{K}-1} \bar{y}_q \log(\hat y_q),$$
if both $\hat y$ and $\bar{y}$ are elements of $[0, 1]^K$.

Complete the implementation of `multiclass_cross_entropy_loss`.

_Points:_ 8

In [ ]:
def multiclass_cross_entropy_loss(Y_pred, Y):
    """
    Compute the cross-entropy loss for multiclass classification.

    Parameters:
    Y_pred (numpy array): Predicted probabilities of shape (n, K) where n is the number of samples and K is the number of classes.
    Y (numpy array): True labels (0, 1, 2) of shape (n,).

    Returns:
    loss (numpy array): Cross-entropy loss of shape (n,).
    """
    # Clip predictions to avoid log(0)
    Y_pred = np.clip(Y_pred, 1e-15, 1 - 1e-15)

    ### YOUR CODE HERE ###
    if not np.issubdtype(Y.dtype, np.integer):
      Y = Y.astype(int)


    correct_class_probs = Y_pred[np.arange(Y_pred.shape[0]), Y]
    loss = -np.log(correct_class_probs)


    ######################

    return loss

In [ ]:
grader.check("q3_3")

q3_3 results: All test cases passed!

### Plotting the Estimated Multiclass Cross-Entropy Loss

Below is a plot of the estimated loss (using the multiclass cross-entropy loss function) for the predictor output by the `bgd_softmax_regression_learner` as a function of the number of epochs.

If your implementation of `bgd_softmax_regression_learner` is correct, you should see the loss decrease as the number of epochs increases.

In [ ]:
# @title Plot

# Function to show/hide the plot
def plot_ce_loss_multiclass(show_plot):
    # Define the number of epochs
    epochs = 200

    # Initialize lists to store the loss values
    cross_entropy_loss_values = []

    # Train the model and compute the loss for each epoch
    for epoch in range(1, epochs + 1):
        predictor, W = bgd_softmax_regression_learner(X_train_norm_mul, Y_train_mul, step_size=0.1, epochs=epoch, random_seed=random_seed)
        loss = estimated_loss(predictor, X_train_norm_mul, Y_train_mul, multiclass_cross_entropy_loss)
        cross_entropy_loss_values.append(loss)

    if show_plot:
        # Plot the loss values
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, epochs + 1), cross_entropy_loss_values, linewidth=3)
        plt.xlabel('Epochs')
        plt.ylabel('Estimated Cross-Entropy Loss')
        plt.grid(True)
        plt.show()
    else:
        clear_output()

# Create a checkbox widget
show_plot_checkbox_ce_loss_multiclass = widgets.Checkbox(value=False, description='Show Plot')

# Use interactive_output to link the function with the checkbox
interactive_plot_ce_loss_multiclass = interactive_output(plot_ce_loss_multiclass, {'show_plot': show_plot_checkbox_ce_loss_multiclass})

# Display the checkbox and the plot
display(show_plot_checkbox_ce_loss_multiclass, interactive_plot_ce_loss_multiclass)

Checkbox(value=False, description='Show Plot')

Output()

### Multiclass Classification Learner

Recall that our objective in multiclass classification is to predict the class of a data point as one of the three cultivars.
So far we have implemented the softmax regression learner, which returns a predictor that outputs a proability of the data point being in each of the three classes.
To make a prediction, we need to convert this probability to a class label.
We can do this by selecting the class with the highest probability.
This is implemented for you as `multiclass_classification_learner` below.

In [ ]:
def multiclass_classification_learner(X, Y, step_size=0.01, epochs=10, random_seed=42):
    """
    Trains a multiclass classification model using batch gradient descent and returns a predictor function.

    Parameters:
    X (numpy array): Feature matrix of size (n, d+1), where n is the number of samples
                     and d is the number of features. The first column should be all 1s.
    Y (numpy array): Target vector of size (n,) with class labels.
    step_size (float): The step size for gradient descent.
    epochs (int): The number of iterations to run gradient descent.
    random_seed (int, optional): The seed for the random number generator.

    Returns:
    predictor (function): A function that takes a feature vector or matrix and returns predicted class.
    W (numpy array): The final weights after applying gradient descent for the specified epochs.
    """
    # Use the bgd_softmax_regression_learner to train the model
    predictor, W = bgd_softmax_regression_learner(X, Y, step_size, epochs, random_seed)

    # Define a new predictor function that returns the class with the highest probability
    def multiclass_predictor(x):
        probabilities = predictor(x)
        return np.argmax(probabilities, axis=1)

    return multiclass_predictor, W

### Plotting the Estimated Zero-One Loss

Similar to the binary classification problem, we would like to evaluate the performance of the multiclass classification learner using the zero-one loss function.

If your implementation of `multiclass_cross_entropy_loss` is correct, you should see the zero-one loss decrease as the number of epochs increases in the plot below.
However, it likely will not decrease as smoothly as the cross-entropy loss, since the `bgd_softmax_regression_learner` is not directly minimizing the zero-one loss.

In [ ]:
# @title Plot

# Function to show/hide the plot
def plot_ce_zo_loss_multi(show_plot):
    # Define the number of epochs
    epochs = 200

    # Initialize lists to store the loss values
    cross_entropy_loss_values = []
    zero_one_loss_values = []

    # Train the model and compute the loss for each epoch
    for epoch in range(1, epochs + 1):
        predictor, W = bgd_softmax_regression_learner(X_train_norm_mul, Y_train_mul, step_size=0.1, epochs=epoch, random_seed=random_seed)
        loss = estimated_loss(predictor, X_train_norm_mul, Y_train_mul, multiclass_cross_entropy_loss)
        cross_entropy_loss_values.append(loss)

        # Compute 0-1 loss
        predictor_class, W = multiclass_classification_learner(X_train_norm_mul, Y_train_mul, step_size=0.1, epochs=epoch, random_seed=random_seed)
        zero_one_loss_value = estimated_loss(predictor_class, X_train_norm_mul, Y_train_mul, zero_one_loss)
        zero_one_loss_values.append(zero_one_loss_value)

    if show_plot:
        # Plot the loss values
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, epochs + 1), cross_entropy_loss_values, label='Cross-Entropy Loss', linewidth=3)
        plt.plot(range(1, epochs + 1), zero_one_loss_values, label='Zero-One Loss', linewidth=3)
        plt.xlabel('Epochs')
        plt.ylabel('Estimated Loss')
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        clear_output()

# Create a checkbox widget
show_plot_checkbox_ce_zo_loss_multi = widgets.Checkbox(value=False, description='Show Plot')

# Use interactive_output to link the function with the checkbox
interactive_plot_ce_zo_loss_multi = interactive_output(plot_ce_zo_loss_multi, {'show_plot': show_plot_checkbox_ce_zo_loss_multi})

# Display the checkbox and the plot
display(show_plot_checkbox_ce_zo_loss_multi, interactive_plot_ce_zo_loss_multi)

Checkbox(value=False, description='Show Plot')

Output()

### Plotting a Linear Decision Boundary

The multiclass classification predictor outputs the class with the highest probability.
Since the probability of a class $y \in \cal{Y}$ is given by $\sigma_y(\mathbf{x}^\top \mathbf{w}_y)$, it implies that the class with the highest probability is the class with the highest value of $\mathbf{x}^\top \mathbf{w}_y$.
If we have two different classes $y, y' \in \cal{Y}$, the decision boundary between these two classes is given by the line $\mathbf{x}^\top (\mathbf{w}_y - \mathbf{w}_{y'}) = 0$.
This line represents when the predicted probability of class $y$ is equal to the predicted probability of class $y'$.
Thus, anything to one side of the line will be predicted as class $y$ and anything to the other side will be predicted as class $y'$.

Below we plot this decision boundary for all possible pairs of classes.

In [ ]:
# @title Plot

# Function to show/hide the plot
def plot_db_multi(show_plot):
    # Function to plot the multiclass decision boundary
    def plot_multiclass_decision_boundary(X, Y, f, title, ax):
        # Create a mesh grid
        x_min, x_max = -2.8, 2.8  # Hardcoded x-limits
        y_min, y_max = -2.2, 3.8  # Hardcoded y-limits
        xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                            np.arange(y_min, y_max, 0.01))

        # Compute the decision boundary
        Z = f(np.c_[np.ones((xx.ravel().shape[0], 1)), xx.ravel(), yy.ravel()])
        Z = Z.reshape(xx.shape)

        # Plot the decision boundary
        ax.contourf(xx, yy, Z, alpha=0.8, levels=[-1, 0, 1, 2], colors=['#FFAAAA', '#AAAAFF', '#AAFFAA'])
        ax.scatter(X[Y == 0][:, 1], X[Y == 0][:, 2], color='red', label='Barolo', s=50)
        ax.scatter(X[Y == 1][:, 1], X[Y == 1][:, 2], color='blue', label='Grignolino', s=50)
        ax.scatter(X[Y == 2][:, 1], X[Y == 2][:, 2], color='green', label='Barbera', s=50)
        ax.set_xlabel('Alcohol (normalized)')
        ax.set_ylabel('Malic Acid (normalized)')
        ax.set_title(title)

        # Hardcoded limits to ensure consistency across both plots
        ax.set_xlim(x_min, x_max)
        ax.set_ylim(y_min, y_max)
        ax.legend()

    epochs = 10000

    if show_plot:
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))

        predictor, W = multiclass_classification_learner(X_train_norm_mul, Y_train_mul, step_size=0.1, epochs=epochs, random_seed=random_seed)
        # Plot the decision boundary on the training set
        plot_multiclass_decision_boundary(X_train_norm_mul, Y_train_mul, predictor, 'Training Set', axes[0])

        # Plot the decision boundary on the test set
        plot_multiclass_decision_boundary(X_test_norm_mul, Y_test_mul, predictor, 'Test Set', axes[1])

        plt.show()
    else:
        clear_output()

# Create a checkbox widget
show_plot_checkbox_db_multi = widgets.Checkbox(value=False, description='Show Plot')

# Use interactive_output to link the function with the checkbox
interactive_plot_db_multi = interactive_output(plot_db_multi, {'show_plot': show_plot_checkbox_db_multi})

# Display the checkbox and the plot
display(show_plot_checkbox_db_multi, interactive_plot_db_multi)

Checkbox(value=False, description='Show Plot')

Output()

### Plotting Polynomial Decision Boundaries

Similar to the binary classification problem, we can use polynomial features to allow for more complex decision boundaries in the multiclass classification problem.
Below is a plot of the decision boundary for different polynomial degrees $p$.
You should see that as $p$ increases the decision boundary becomes more complex and can better separate the three classes in the training set.
However, if $p$ is too large, then the decision boundary may become too complex and overfit the training data, leading to poor performance on the test set.

In [ ]:
# @title Plot

# Function to show/hide the plots and update the polynomial degree
def plot_db_multi_poly(show_plot, degree):
    # Define the decision boundary function for polynomial features
    def plot_multiclass_decision_boundary_poly(X, Y, f, degree, title, X_poly_mean=None, X_poly_std=None, ax=None):
        # Create a mesh grid
        x_min, x_max = -2.8, 2.8  # Hardcoded x-limits
        y_min, y_max = -2.2, 3.8  # Hardcoded y-limits
        xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                            np.arange(y_min, y_max, 0.01))

        # Transform the mesh grid to polynomial features
        poly_features = phi_p(np.c_[xx.ravel(), yy.ravel()], degree)

        # Normalize the polynomial features using the same mean and std as the training set
        poly_features = (poly_features - X_poly_mean) / X_poly_std

        # Append a column of 1s to the features for the bias term
        poly_features = np.hstack([np.ones((poly_features.shape[0], 1)), poly_features])

        # Compute the decision boundary
        # Z = poly_features.dot(W)
        # Z = np.argmax(Z, axis=1)
        Z = f(poly_features)
        Z = Z.reshape(xx.shape)

        # Plot the decision boundary
        ax.contourf(xx, yy, Z, alpha=0.8, levels=[-1, 0, 1, 2], colors=['#FFAAAA', '#AAAAFF', '#AAFFAA'])
        ax.scatter(X[Y == 0][:, 1], X[Y == 0][:, 2], color='red', label='Barolo', s=50)
        ax.scatter(X[Y == 1][:, 1], X[Y == 1][:, 2], color='blue', label='Grignolino', s=50)
        ax.scatter(X[Y == 2][:, 1], X[Y == 2][:, 2], color='green', label='Barbera', s=50)
        ax.set_xlabel('Alcohol (normalized)')
        ax.set_ylabel('Malic Acid (normalized)')
        ax.set_ylim(y_min, y_max)
        ax.set_xlim(x_min, x_max)
        ax.set_title(title)
        ax.legend()

    epochs = 10000

    if show_plot:
        # Transform the input features to polynomial features of the selected degree
        X_poly_train = phi_p(X_train_norm_mul[:, 1:], degree)
        X_poly_test = phi_p(X_test_norm_mul[:, 1:], degree)

        # Normalize the polynomial features
        X_poly_mean = X_poly_train.mean(axis=0)
        X_poly_std = X_poly_train.std(axis=0) + 1e-8
        X_poly_train = (X_poly_train - X_poly_mean) / X_poly_std
        X_poly_test = (X_poly_test - X_poly_mean) / X_poly_std

        # Append a column of 1s to the features for the bias term
        X_poly_train = np.hstack([np.ones((X_poly_train.shape[0], 1)), X_poly_train])
        X_poly_test = np.hstack([np.ones((X_poly_test.shape[0], 1)), X_poly_test])

        # Train the model using polynomial features
        predictor_poly, w_poly = multiclass_classification_learner(X_poly_train, Y_train_mul, step_size=0.1, epochs=epochs, random_seed=random_seed)

        fig, axes = plt.subplots(1, 2, figsize=(15, 6))

        # Plot the decision boundary on the training set
        plot_multiclass_decision_boundary_poly(X_poly_train, Y_train_mul, predictor_poly, degree, f'Training Set (Polynomial Degree {degree})', X_poly_mean, X_poly_std, axes[0])

        # Plot the decision boundary on the test set
        plot_multiclass_decision_boundary_poly(X_poly_test, Y_test_mul, predictor_poly, degree, f'Test Set (Polynomial Degree {degree})', X_poly_mean, X_poly_std, axes[1])

        plt.show()
    else:
        clear_output()

# Create a checkbox widget
show_plot_checkbox_db_multi_poly = widgets.Checkbox(value=False, description='Show Plot')

# Create a slider widget for selecting the polynomial degree
degree_slider_db_multi_poly = widgets.IntSlider(value=1, min=1, max=10, step=0.5, description='Degree')

# Use interactive_output to link the function with the checkbox and slider
interactive_plot_db_multi_poly = interactive_output(plot_db_multi_poly, {'show_plot': show_plot_checkbox_db_multi_poly, 'degree': degree_slider_db_multi_poly})

# Display the checkbox, slider, and the plot
display(show_plot_checkbox_db_multi_poly, degree_slider_db_multi_poly, interactive_plot_db_multi_poly)

Checkbox(value=False, description='Show Plot')

IntSlider(value=1, description='Degree', max=10, min=1, step=0)

Output()

To select the best predictor we should plot the estimated zero-one loss on the test set as a function of the polynomial degree $p$.
The value of $p$ with the lowest zero-one loss on the test set is the best predictor.

In the plot below you should see that values of $p=7, 8$ have the lowest estimated zero-one loss on the test set.

In [ ]:
# @title Plot

# Function to show/hide the plot
def plot_loss_vs_deg_multi(show_plot):
    epochs = 10000
    # Define the range of polynomial degrees
    degrees = range(1, 11)

    # Initialize lists to store the loss values
    train_zero_one_loss_values_by_degree = []
    test_zero_one_loss_values_by_degree = []

    # Train the model and compute the loss for each polynomial degree
    for degree in degrees:
        # Transform the input features to polynomial features of the selected degree
        X_poly_train = phi_p(X_train_norm_mul[:, 1:], degree)
        X_poly_test = phi_p(X_test_norm_mul[:, 1:], degree)

        # Normalize the polynomial features
        X_poly_mean = X_poly_train.mean(axis=0)
        X_poly_std = X_poly_train.std(axis=0) + 1e-8
        X_poly_train = (X_poly_train - X_poly_mean) / X_poly_std
        X_poly_test = (X_poly_test - X_poly_mean) / X_poly_std

        # Append a column of 1s to the features for the bias term
        X_poly_train = np.hstack([np.ones((X_poly_train.shape[0], 1)), X_poly_train])
        X_poly_test = np.hstack([np.ones((X_poly_test.shape[0], 1)), X_poly_test])

        # Train the model using polynomial features
        predictor_poly, w_poly = multiclass_classification_learner(X_poly_train, Y_train_mul, step_size=0.1, epochs=epochs, random_seed=random_seed)

        # Compute the train and test 0-1 loss
        train_zero_one_loss_value = estimated_loss(predictor_poly, X_poly_train, Y_train_mul, zero_one_loss)
        test_zero_one_loss_value = estimated_loss(predictor_poly, X_poly_test, Y_test_mul, zero_one_loss)

        # Store the loss values
        train_zero_one_loss_values_by_degree.append(train_zero_one_loss_value)
        test_zero_one_loss_values_by_degree.append(test_zero_one_loss_value)

    if show_plot:
        plt.figure(figsize=(10, 6))
        plt.plot(degrees, train_zero_one_loss_values_by_degree, label='Train Zero-One Loss', linewidth=3)
        plt.plot(degrees, test_zero_one_loss_values_by_degree, label='Test Zero-One Loss', linewidth=3)
        plt.xlabel('Polynomial Degree')
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        clear_output()

# Create a checkbox widget
show_plot_checkbox_loss_vs_deg_multi = widgets.Checkbox(value=False, description='Show Plot')

# Use interactive_output to link the function with the checkbox
interactive_plot_loss_vs_deg_multi = interactive_output(plot_loss_vs_deg_multi, {'show_plot': show_plot_checkbox_loss_vs_deg_multi})

# Display the checkbox and the plot
display(show_plot_checkbox_loss_vs_deg_multi, interactive_plot_loss_vs_deg_multi)

Checkbox(value=False, description='Show Plot')

Output()

# The End :)

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

q2_1 results: All test cases passed!

q2_2 results: All test cases passed!

q2_3 results: All test cases passed!

q2_4 results: All test cases passed!

q2_5 results: All test cases passed!

q3_1 results: All test cases passed!

q3_2 results: All test cases passed!

q3_3 results: All test cases passed!